<a href="https://colab.research.google.com/github/nferrucho/NPL/blob/main/curso3/ciclo1/1_kdd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://drive.google.com/uc?export=view&id=1li4ahmMhPo2cEUVqQKRDA9ahHp2py4Xb" width="100%">

# Knowledge Discovery in Databases
---

En este notebook veremos un ejemplo práctico de la metodología _Knowledge Discovery in Databases_ (KDD) descrita en el siguiente diagrama:

<img src="https://drive.google.com/uc?export=view&id=1Dyi9xBZp9ohTFw9pUGRjbUwlz2ihQjRA" width="80%">

Este problema lo abordaremos con las siguientes librerías:

> **Nota**: como puede observar, usaremos `pandas` y `numpy` para manipulación de datos, `matplotlib` y `seaborn` para visualización de datos, y `statsmodels` para modelamiento. Es importante tener en cuenta esto, ya que uno de los problemas que vamos a abordar a lo largo de este curso es saber cómo podemos integrar proyectos de machine learning independientemente de las librerías o el lenguaje de programación usado.

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

## **1. Contexto**
---

En este caso, estaremos trabajando con el conjunto de datos [customer personality analysis](https://www.kaggle.com/imakash3011/customer-personality-analysis), el cual contiene información demográfica de ventas y promociones sobre clientes de una empresa:

<img src="https://drive.google.com/uc?export=view&id=11CGnstfN7q1vrl9abTgdGEIPU6kumQtn" width="80%">

En este caso tenemos como objetivo determinar si un cliente aceptará o no una campaña publicitaria dadas sus características:

<img src="https://drive.google.com/uc?export=view&id=1wjbWqo6YGCT-VtXZHYGIO5NrK1Vx5TtB" width="80%">

Este conjunto de datos contiene los siguientes atributos y campos:

**Información sociodemográfica**:

- `ID`: Identificador único del cliente.
- `Year_Birth`: Año de nacimiento.
- `Education`: Nivel de estudios del cliente.
- `Marital_Status`: Estado civil.
- `Income`: ingresos anuales del cliente.
- `Kidhome`: Número de hijos menores del cliente
- `Teenhome`: Número de hijos adolescentes del cliente.
- `Dt_Customer`: fecha de inscripción del cliente con la empresa.
- `Recency`: número de días desde la última compra.
- `Complain`: específica si el cliente ha realizado algún reclamo en los últimos dos años.

**Información de ventas**:

- `MntWines`: cantidad gastada en vinos en los últimos 2 años.
- `MntFruits`: cantidad gastada en frutas en los últimos 2 años.
- `MntMeatProducts`: cantidad gastada en carnes en los últimos 2 años.
- `MntFishProducts`: cantidad gastada en pescados en los últimos 2 años.
- `MntSweetProducts`: cantidad gastada en dulces en los últimos 2 años.
- `MntGoldProds`: cantidad gastada en productos con oro en los últimos 2 años.

**Información sobre promociones**:

- `NumDealsPurchases`: número de compras realizadas con descuento.
- `AcceptedCmp1`: específica si el cliente aceptó la primera campaña publicitaria.
- `AcceptedCmp2`: específica si el cliente aceptó la segunda campaña publicitaria.
- `AcceptedCmp3`: específica si el cliente aceptó la tercera campaña publicitaria.
- `AcceptedCmp4`: específica si el cliente aceptó la cuarta campaña publicitaria.
- `AcceptedCmp5`: específica si el cliente aceptó la quinta campaña publicitaria.
- `Response`: específica si el cliente aceptó la última campaña publicitaria.

**Información sobre medios de compra**:

- `NumWebPurchases`: número de compras realizadas a través de la página web de la empresa.
- `NumCatalogPurchases`: número de compras realizadas a través de un catálogo telefónico.
- `NumStorePurchases`: número de compras realizadas directamente en las tiendas físicas.
- `NumWebVisitsMonth`: número de visitas a la página web de la empresa en el último mes.

Comenzamos cargando el conjunto de datos:

In [ ]:
data = pd.read_csv("https://raw.githubusercontent.com/mindlab-unal/mlds6-datasets/main/u1/marketing_campaign.csv", sep="\t")
display(data.head())

Veamos paso a paso, cómo podemos entrenar un modelo para determinar qué usuarios podrían aceptar una promoción según la metodología _KDD_.

## **2. Selección**
---

La metodología _KDD_ formalmente inicia desde este paso, es decir, se asume que el científico de datos ya tiene conocimientos sobre el negocio y los datos.

<img src="https://drive.google.com/uc?export=view&id=1StofJqVJ6pxfIopijtp37PeftPxOQdJZ" width="80%">

Podemos hacer una inspección rápida del conjunto de datos, como el tamaño:

In [ ]:
display(data.shape)

También podemos obtener información sobre los campos y tipos del dataset:

In [ ]:
display(data.dtypes)

Este conjunto de datos está conformado por 28 columnas, de las cuales:

- Hay columnas de tipo entero como el año de nacimiento `Year_Birth`, el número de hijos menores de 14 años `Kidhome`, entre otros.
- Algunas variables son de tipo cadena de caracteres como la educación `Education`, el estado marital `Marital_Status`, entre otros. Estas variables las trataremos como variables nominales.
- Otras variables son numéricas como el ingreso del cliente `Income`.
- La variable a predecir `Response` contiene valores binarios.
- Hay columnas como el `ID` y variables desconocidas (`Z_CostContact`, `Z_Revenue`) que no deberían ser incluidas en el modelo.

El proceso de **selección** consiste en seleccionar los registros y las columnas que necesitaremos para nuestra solución de analítica.

Para esto, comenzaremos validando si hay valores faltantes en el conjunto de datos:

In [ ]:
display(data.isnull().sum())

Como podemos ver, la variable `Income` contiene 24 valores faltantes. Procedemos a eliminarlos:

In [ ]:
selected_rows = data.dropna()
display(data.shape[0])
display(selected_rows.shape[0])

Como podemos ver, `selected_rows` es un `DataFrame` donde se eliminaron las filas con valores faltantes.

Ahora, vamos a seleccionar las columnas necesarias para la aplicación, para ello, vamos a realizar dos filtros:

- Eliminamos columnas que no deseamos tener como variables de entrada al modelo.
- Eliminamos todas las columnas que estén muy correlacionadas con la variable objetivo, por ejemplo, aquellas que indican si un cliente aceptó otro tipo de promoción ya que no sabemos en qué fecha se realizó cada una (las filtramos con una expresión regular que excluye nombres de columnas donde haya al menos un número).

In [ ]:
selected_variables = (
        selected_rows
        .drop(columns=["ID", "Z_CostContact", "Z_Revenue"])
        .filter(regex=r"^[a-zA-Z_]+$")
        )
display(selected_variables.columns)

In [ ]:
selected_variables

## **3. Preprocesamiento**
---

La etapa de preprocesamiento en _KDD_ consiste en modificar los datos del conjunto de datos para dejarlos en un formato un poco más estándar, creación de nuevas variables, unión o división de categorías.

La etapa de preprocesamiento parte de los datos seleccionados y permite llegar a una versión preprocesada de los datos:

<img src="https://drive.google.com/uc?export=view&id=1q0X6cP0l2bmLFAkpDIkF23BWx2lNnelG" width="80%">

En nuestro caso, vamos a modificar algunas columnas para que se puedan manipular más fácilmente.

Primero, usaremos la columna `Year_Birth` para calcular la edad de las personas. Esto fundamentalmente por temas de magnitud y estabilidad numérica en los modelos (es más fácil manipular el número 25 en lugar de el año 1996).

Para esto, definimos la función `get_age` para hacer el cálculo con la librería `datetime` para el manejo de fechas.

In [ ]:
import datetime as dt

La función nos permite obtener la diferencia entre el año tiempo actual `utcnow()` (formato UTC) y el año almacenado en el conjunto de datos:

In [ ]:
def get_age(df):
    return dt.datetime.utcnow().year - df["Year_Birth"]

Con esta función podemos calcular el número de años que tiene el cliente. Veamos un ejemplo, primero calculamos la fecha actual:

In [ ]:
display(dt.datetime.utcnow())

Veamos la diferencia en años de alguien que nació en 1985 (aproximado de su edad):

In [ ]:
display(dt.datetime.utcnow().year - 1985)

De la misma forma, podemos calcular cuántos años de antigüedad tiene el cliente en la compañía. Como la información es un poco más precisa (disponemos de fechas), podemos calcular los años con cifras decimales con la función `gen_antiquity`:

In [ ]:
def get_antiquity(df):
    return (
            dt.datetime.utcnow() -
            pd.to_datetime(df["Dt_Customer"], format="%d-%m-%Y")
            ).dt.days / 365

Finalmente, creamos el conjunto de datos preprocesado al reemplazar las variables antiguas por las preprocesadas:

In [ ]:
preprocessed_data = (
        selected_variables
        .assign(
            age = get_age, antiquity = get_antiquity
            )
        .drop(columns=["Dt_Customer", "Year_Birth"])
        )
display(preprocessed_data.columns)

El método `assign` de `pandas` nos permite crear nuevas columnas a partir de funciones (esto ayuda a estructurar mejor el código). Veamos una descripción de las nuevas columnas que calculamos:

In [ ]:
display(
        preprocessed_data
        .filter(["age", "antiquity"])
        .describe()
        )

In [ ]:
preprocessed_data

## **4. Transformación**
---

El proceso de transformación en _KDD_ parte de los datos preprocesados y permite extraer datos transformados:

<img src="https://drive.google.com/uc?export=view&id=19iYXSMlPIMcxsEnSJwRrmFOFU_A8ZfTq" width="80%">

En términos más simples, el proceso de transformación consiste en extraer características o representaciones puramente numéricas, para dejar listas las entradas y salidas del modelo. En especial, como vamos a estar trabajando en un problema supervisado, vamos a comenzar separando las variables explicativas de la variable objetivo:

In [ ]:
variables = preprocessed_data.drop(columns=["Response"])
target = preprocessed_data.Response.values

Ahora, sobre las variables explicativas `variables` sabemos que tenemos una mezcla de distintos tipos de variables, por ello, debemos plantearnos las siguientes preguntas:

1. ¿Qué tipo de tratamiento se le puede dar a una variable numérica?

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Respuesta</b></font>
</summary>

Existen distintos tratamientos que podemos realizar en variables numéricas, entre ellos:

- Normalización con respecto a alguna norma (Euclidiana, Manhattan, entre otras).
- Z-scaling, es decir, eliminación de medias y desviaciones estándar por variable.
- Min-Max, es decir, acotar el rango de los datos a una escala fija.
</details>

2. ¿Qué tipo de tratamiento se le puede dar a una variable ordinal?

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Respuesta</b></font>
</summary>

Existen distintos tratamientos que podemos realizar en variables ordinales, entre ellos:

- Se puede utilizar una codificación ordinal, es decir, en el mismo orden de las variables asignar números enteros.
- Cuando las variables ordinales no tienen muchos valores posibles, podemos manejarlas como variables categóricas.
- Cuando las variables ordinales tienen muchos valores posibles, se pueden manipular como variables numéricas.
</details>

3. ¿Qué tipo de tratamiento se le puede dar a una variable categórica?

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Respuesta</b></font>
</summary>

Existen distintos tratamientos que podemos realizar en variables categóricas, entre ellos:

* La codificación de tipo one-hot o la creación de variables _dummy_.
* Target encoding, es decir, reemplazar cada categoría con el resultado de un descriptivo (por ejemplo, el promedio de variables numéricas de la categoría, conteos, entre otros).
* _Embeddings_, como normalmente se realiza en procesamiento de lenguaje natural o con modelos de _Deep Learning_.
</details>

Para la transformación, vamos a definir los siguientes tipos de variables:

In [ ]:
categorical = ["Education", "Marital_Status"]
ordinal = ["Kidhome", "Teenhome", "Recency", "age"]
numeric = ["Income", "antiquity"]
binary = ["Complain"]

Ahora, vamos a definir algunas funciones para transformar estas variables en números dependiendo del tipo de dato que tengamos.

* Para las variables categóricas calculamos variables _dummy_:

In [ ]:
def get_dummies(df, cols):
    dummies = []
    for col in cols:
        dummies.append(pd.get_dummies(df[col]))
    return pd.concat(dummies, axis=1).values

* Para las variables ordinales y numéricas, realizamos un reescalamiento de tipo MinMax:

In [ ]:
def min_max(df, cols):
    data = df.filter(cols).values
    transformed_data = (
            (data - data.min(keepdims=True, axis=0)) /
            (data.max(keepdims=True, axis=0) - data.min(keepdims=True, axis=0))
            )
    return transformed_data

* La variable binaria, la vamos a dejar tal cual viene:

In [ ]:
def identity(df, cols):
    return df.filter(cols).values

Finalmente, extraemos todas las características y las concatenamos en una única representación:

In [ ]:
categorical_features = get_dummies(preprocessed_data, categorical)
ordinal_features = min_max(preprocessed_data, ordinal)
numeric_features = min_max(preprocessed_data, numeric)
binary_features = min_max(preprocessed_data, binary)
features = np.concatenate(
    [categorical_features, ordinal_features, numeric_features, binary_features],
    axis=1
)
display(features.shape)

In [ ]:
features

Como puede ver, nos quedamos con una representación de 20 características.

## **5. Minería de Datos**
---

La etapa de minería de datos consiste en entrenar un modelo de machine learning sobre las características extraídas:

<img src="https://drive.google.com/uc?export=view&id=1SfYgZZfoX6fcontSzTmT4ljqsCO-RJwD" width="80%">

Primero, vamos a aplicar una estrategia de muestreo `SMOTE` para balancear las etiquetas:

In [ ]:
display(np.unique(target, return_counts=True))

In [ ]:
from imblearn.over_sampling import SMOTE

Balanceamos el conjunto de datos:

In [ ]:
features, target = SMOTE(random_state=0).fit_resample(features, target)

Podemos validar que las etiquetas se encuentran balanceadas:

In [ ]:
display(np.unique(target, return_counts=True))

Para esto, vamos a entrenar un modelo de regresión logística desde `statsmodels`, comenzamos definiendo el modelo:

In [ ]:
model = sm.Logit(endog=target, exog=features)

Entrenamos el modelo:

In [ ]:
results = model.fit()

## **6. Evaluación**
---

Por último, la evaluación consiste en evaluar algunas métricas y en la posibilidad de utilizar los patrones encontrados:

<img src="https://drive.google.com/uc?export=view&id=1YNAo3g8RHqI-mSvTVFDpF70XjN570IJr" width="80%">

Para ello, primero veremos las métricas de desempeño del modelo:

In [ ]:
display(results.summary())

Podemos ver métricas de desempeño como `Log-likelihood` (entre más cercano a cero mejor) y otra información estadística de los parámetros del modelo.

Ahora, podemos generar predicciones del modelo:

In [ ]:
preds = results.predict(features)
display(preds)

Como podemos ver, el resultado son valores entre 0 y 1. Veamos una comparativa de las predicciones del modelo y los valores reales de la variable dependiente, para ello construimos el siguiente `DataFrame`:

In [ ]:
preds = pd.DataFrame({"target": target, "preds": preds})
display(preds)

Generamos una gráfica de distribuciones con `seaborn`

In [ ]:
fig, ax = plt.subplots()
sns.histplot(preds, x="preds", hue=target, kde=True)
fig.show()

Como puede ver las predicciones corresponden mayoritariamente a la clase correspondiente, aunque existe una región de incertidumbre (valores cercanos a 0) que pueden llegar a ser de interés para un futuro análisis.

## Recursos Adicionales
---

Los siguientes enlaces corresponden a sitios donde encontrará información muy útil para profundizar en los temas vistos en este notebook:

- [Statsmodels](https://www.statsmodels.org/stable/index.html)
- [Matplotlib](https://matplotlib.org/)
- [Seaborn](https://seaborn.pydata.org/)

## Créditos
---

**Profesor**

- [Jorge E. Camargo, PhD](https://dis.unal.edu.co/~jecamargom/)

**Asistente docente**:

- [Juan S. Lara MSc](https://www.linkedin.com/in/juan-sebastian-lara-ramirez-43570a214/)

**Diseño de imágenes:**
- [Brian Chaparro Cetina](mailto:bchaparro@unal.edu.co).

**Universidad Nacional de Colombia** - *Facultad de Ingeniería*